# Исследование надежности заемщиков

## Откроем таблицу и изучим общую информацию о данных

In [31]:
# импортируем библиотеку pandas
import pandas as pd

In [32]:
# прочитаем csv-файл
data=pd.read_csv('data.csv')

### Первые 20 строчек датафрейма data на экран.

In [33]:
# ваш код здесь
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


#### Основная информация о датафрейме с помощью метода info()

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

#### Количество пропущенных значений для каждого столбца.

In [35]:
print(data.isnull().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


#### В двух столбцах есть пропущенные значения. Один из них — days_employed. Другой столбец с пропущенными значениями — total_income — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца income_type. Например, у человека с типом занятости сотрудник пропуск в столбце total_income должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [36]:
median_income = data.groupby('income_type')['total_income'].transform('median') 
print(data['total_income'].fillna(median_income, inplace=True))

None


### Обработка аномальных значений

#### В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце days_employed. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.

In [37]:
data['days_employed']=abs(data['days_employed'])

#### Для каждого типа занятости выведим медианное значение трудового стажа days_employed в днях.

In [38]:
print(data.groupby('income_type')['days_employed'].median())

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64


#### Выведим перечень уникальных значений столбца `children`.

In [39]:
print(data['children'].unique())

[ 1  0  3  2 -1  4 20  5]


#### В столбце children есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма data.

In [40]:
data = data.loc[data['children'] !=-1]
data = data.loc[data['children'] !=20]

#### Ещё раз выведим перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.

In [41]:
print(data['children'].unique())

[1 0 3 2 4 5]


### Удаление пропусков (продолжение)

####  Заполним пропуски в столбце days_employed медианными значениями по каждому типу занятости income_type

In [42]:
import warnings
warnings.filterwarnings("ignore")

for type in data['income_type'].unique():
    data['days_employed'].loc[(data['income_type'] == type) & (data['days_employed'].isna() == True)] = data['days_employed'].loc[data['income_type'] == type].median()

#### Убедимся, что все пропуски заполнены. Проверbv себя и ещё раз выведиv количество пропущенных значений для каждого столбца с помощью двух методов.

In [43]:
print(data.isna().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


### Изменение типов данных

#### Заменим вещественный тип данных в столбце total_income на целочисленный с помощью метода astype().

In [44]:
data['total_income'] = data['total_income'].astype('int')

#### Обработка дубликатов

#### Обработаем неявные дубликаты в столбце education. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. 

In [45]:
data['education'] = data['education'].str.lower()

#### Выведим на экран количество строк-дубликатов в данных и удалим их

In [46]:
data.duplicated().sum()

71

In [47]:
data = data.drop_duplicates()

### Категоризация данных

#### На основании диапазонов, указанных ниже, создадим  в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [48]:
def categorize_income(income):
    if income < 30001:
        return 'E'
    elif 30001 <= income < 50001:
        return 'D'
    elif 50001 <= income <200001:
        return 'C'
    elif 200001 <= income <1000001:
        return 'B'
    else:
        return 'A'

In [49]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

#### Выведим на экран перечень уникальных целей взятия кредита из столбца `purpose`

In [50]:
print(data['purpose'].unique())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


#### Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:

- 'операции с автомобилем',
- 'операции с недвижимостью',
- 'проведение свадьбы',
- 'получение образования'

In [51]:
def categorize_purpose(str):
    if 'авто' in str:
        return 'операции с автомобилем'
    elif 'жиль' in str or 'движ' in str:
        return 'операции с недвижимостью'
    elif 'свадьб' in str:
        return 'проведение свадьбы'
    elif 'образ' in str:
        return 'получение образования'
    else:
        return 'прочее'

In [52]:
# применим функцию методом apply()
data['purpose_category'] = data['purpose'].apply(categorize_purpose)